In [107]:
import numpy as np
import pandas as pd
import math
from scipy.stats import norm
import matplotlib as plt

# European Option Pricing
## Black-Sholes Model

$$ C = S_0N(d_1)−Ke^{−rT}N(d_2) $$
$$ P = Ke^{−rT}N(-d_2) - S_0N(-d_1) $$

Where:

*   C is the call option price.
*   $S_0$ is the current stock price.
*   K is the option strike price.
*   r is the risk-free interest rate.
*   T is the time to expiration (in years).
*   N() is the cumulative distribution function of the standard normal distribution.
*   $ d_1 = \frac{ln⁡(S_0K)+(r+\sigma^2/2)T}{\sigma\sqrt{T}} $
*   $ d_2 = d1−\sigma\sqrt{T} $
*   $\sigma$ is the volatility of the underlying stock.

In [94]:
def BS_European(S,K,r,sigma,T):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    put_price =  K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    
    index = ['European Call', 'European Put']
    columns = ['Price']
    
    price = pd.DataFrame([call_price, put_price], index = index, columns=columns)
    return price



In [95]:
S = 10000
K = 12000
r = 0.05
sigma = 0.2
T = 1

BS_European(S,K,r,sigma,T)

,Price
European Call,324.747742
European Put,1739.500836


### Greeks

#### Delta
$\Delta_{Call}  = N(d_1) $ 

$\Delta_{Put} = -N(-d_1)$

### Gamma
$ \Gamma = \frac{N'(d_1)}{S\sigma\sqrt{T}} $

### Vega
$ \nu = SN′(d1)T $

### Theta
$ \Theta_{Call} = -\frac{S\sigma N'(d_1)}{2\sqrt{T}} - rKe^{-rT}N(d_2)$ 

$ \Theta_{Put} = -\frac{S\sigma N'(d_1)}{2\sqrt{T}} + rKe^{-rT}N(-d_2)$

### Rho
$ \rho_{Call} = KTe^{-rT}N(d_2) $

$ \rho_{Put} = -KTe^{-rT}N(-d_2) $

In [96]:
def BS_Greeks(S,K,r,sigma,T) :
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    delta_call = norm.cdf(d1)
    delta_put = -norm.cdf(-d1)
    
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    
    vega = S * norm.pdf(d1) * T
    
    theta_call = (-S * sigma * norm.pdf(d1)) / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * norm.cdf(d2)
    theta_put = (-S * sigma * norm.pdf(d1)) / (2 * np.sqrt(T)) + r * K * np.exp(-r * T) * norm.cdf(-d2)
    
    rho_call = K * T * np.exp(-r * T) * norm.cdf(d2) 
    rho_put = -K * T * np.exp(-r * T) * norm.cdf(-d2)
    
    greeks_call = delta_call, gamma, vega, theta_call, rho_call
    greeks_put = delta_put, gamma, vega, theta_put, rho_put
    
    columns = ['Delta', 'Gamma', 'Vega', 'Theta', 'Rho']
    index = ['European Call', 'European Put']
    
    greeks_df = pd.DataFrame([greeks_call,greeks_put], columns = columns, index=index)
    
    return greeks_df


In [97]:
BS_Greeks(S,K,r,sigma,T)

,Delta,Gamma,Vega,Theta,Rho
European Call,0.287192,0.00017,3407.384228,-468.096855,2547.168637
European Put,-0.712808,0.00017,3407.384228,102.640800,-8867.584457


## Monte Carlo Simulation

In [98]:
import numba

In [103]:
def MS_European(S,K,r,sigma,T,num_simulations=10000000):
    
    z = np.random.standard_normal(size=num_simulations)
    
    price_path = S * np.exp((r - 0.5 * sigma**2)* T + sigma * np.sqrt(T) * z)
    
    call_price = np.mean(np.maximum(price_path - K, 0)) * np.exp(-r * T)
    put_price = np.mean(np.maximum(K - price_path, 0)) * np.exp(-r * T)
   
    index = ['European Call', 'European Put']
    columns = ['Price']
    
    price = pd.DataFrame([call_price, put_price], index = index, columns=columns)
    return price

In [104]:
MS_European(S,K,r,sigma,T,num_simulations=10000000)

,Price
European Call,325.317831
European Put,1739.034118


## Finite Differece Method
### Explicit Finite Difference Method

### Implicit Finite Difference Method

### 
